# Effectiveness - Delete a single point


In [ ]:
import sys
sys.path.append("../..")

from sklearn import svm
import numpy as np
from copy import deepcopy

import src

from src.data_utils import (
    load_npy, save_npy, preprocess_data, variance, normalize, generate_union_description
)

In [ ]:
model = svm.SVC(decision_function_shape='ovo')
x_train, y_train, x_test, y_test, _ = preprocess_data('train_100p_3c.csv', 'test_100p_3c.csv')
union_d = generate_union_description(len(x_train), 10)

mc_plus_ov_99 = src.mc_owen(x_train[:99], y_train[:99], x_test, y_test, model, union_d, m=99*1000)
save_npy('mc_plus_ov_99.npy', mc_plus_ov_99)

mc_ov_99 = src.mc_owen(x_train[:99], y_train[:99], x_test, y_test, model, union_d, m=99*100)
save_npy('mc_ov_99.npy', mc_plus_ov_99)

In [ ]:
# delta
init_ov = load_npy('mc_plus_ov_100.npy')

delta_shap = src.DeltaOwen(x_train, y_train, x_test, y_test, model,
                                init_ov)
delta_ov_99 = delta_shap.del_single_point(99, 100*(len(y_train)-1))

save_npy('delta_ov_99.npy', delta_ov_99)

In [ ]:
model = svm.SVC(decision_function_shape='ovo')
init_ov = load_npy('mc_plus_ov_100.npy')

# KNN
heur_shap = src.HeurOwen(x_train, y_train,
                              x_test, y_test,
                              model, init_ov, union_d,
                              params={'method': 'knn'})
heur_shap.prepare(params={'n_neighbors': 3})
knn_ov_99 = heur_shap.del_single_point(99)
save_npy('knn_ov_99.npy', knn_ov_99)

# for simi
model = svm.SVC(decision_function_shape='ovo')

idxs = [18, 56, 89]
ovs = list()

for idx in idxs:
    new_union_d = deepcopy(union_d)
    for _ in new_union_d:
        if idx in _:
            _.remove(idx)
    ov = src.mc_owen(np.delete(x_train, idx, axis=0),
                          np.delete(y_train, idx),
                          x_test, y_test,
                          model, new_union_d, 1000*99, 4)
    save_npy('mc_plus_ov_99_del'+str(idx)+'.npy', ov)
    ovs.append(ov)

ovs = np.asarray(ovs)

save_npy('knn_plus_ovs.npy', ovs)

# KNN+
heur_shap = src.HeurOwen(x_train, y_train,
                              x_test, y_test,
                              model, init_ov, union_d,
                              params={'method': 'knn+'})

heur_shap.prepare(flags={'exact': False, 'train': False},
                  params={'n_neighbors': 3,
                          'simi_type': 'ed',
                          'f_shap': 'n*n',
                          'rela': ['poly', 1],
                          'train_idxs': [18, 56, 89],
                          'train_ovs': load_npy('knn_plus_ovs.npy')})

knn_plus_ov_99 = heur_shap.del_single_point(99)
save_npy('knn_plus_ov_99.npy', knn_plus_ov_99)

In [ ]:
# comparison
def comp(base_v, comp_ov, name):
    var = variance(base_v, normalize(base_v, comp_ov))
    print("variance of %s   \t : %.10f" % (name, var))
    

In [ ]:
comp(mc_plus_ov_99, load_npy('delta_ov_99.npy'), 'delta')
comp(mc_plus_ov_99, load_npy('knn_ov_99.npy'), 'knn')
comp(mc_plus_ov_99, load_npy('knn_plus_ov_99.npy'), 'knn+')
comp(mc_plus_ov_99, load_npy('mc_ov_99.npy'), 'mc')